In [ ]:
%pip install charset-normalizer  # pandas python-dotenv transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from huggingface_hub import login
from dotenv import load_dotenv

import pandas as pd

import torch
import os

c:\Compilers\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load environment variables from .env file
load_dotenv()

token = os.getenv("HUGGINGFACE_API_TOKEN")

login(token)

In [ ]:
model_id = "meta-llama/Llama-3.2-1B"

pipe = pipeline(
    "text-generation", 
    model=model_id, 
    torch_dtype=torch.bfloat16,
    max_length=50
)

response = pipe("What is the most common eaten foodstuff in USA, Los Angeles?", return_full_text=False)
# TODO: pogledat pipe, zasto se dogada repeatanje
response

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
from charset_normalizer import detect

# Read a sample of the file
with open('data/Foodex1.csv', 'rb') as file:
    raw_data = file.read()

# Detect encoding
result = detect(raw_data)
print(f"Detected encoding: {result['encoding']}")

encoding = result['encoding']

Detected encoding: None


In [27]:
# Data scouting

# ISTO JAKO DOBAR DATASET
# harvard_dataset = pd.read_csv("data/edible_food_2011.csv")
# # harvard_dataset.columns.tolist()
# print(len(harvard_dataset['Unnamed: 1'].unique().tolist())) # Ima podosta dobrih podataka za 177 drzava

# ZA SADA NAJBOLJI:
faostat = pd.read_csv("data/FAOSTAT_food_consumption.csv")
# print(faostat['Item'].unique())
# faostat[faostat['Area'] == 'Afghanistan'].head(50)

faostat_filtered_units = faostat[faostat['Unit'] == '1000 t']

faostat_filtered_units = faostat_filtered_units[faostat_filtered_units['Value'] > 0]

unique_countries = faostat_filtered_units['Area'].unique()

for country in unique_countries:
    country_data = faostat_filtered_units[faostat_filtered_units['Area'] == country]

    most_eaten_food = country_data[country_data['Value'] == country_data['Value'].max()]['Item'].values[0]

    print(f"In {country} the most eaten food is {most_eaten_food}")

In Afghanistan the most eaten food is Wheat and products
In Albania the most eaten food is Milk - Excluding Butter
In Algeria the most eaten food is Wheat and products
In Angola the most eaten food is Cassava and products
In Antigua and Barbuda the most eaten food is Fruits, other
In Argentina the most eaten food is Maize and products
In Armenia the most eaten food is Milk - Excluding Butter
In Australia the most eaten food is Milk - Excluding Butter
In Austria the most eaten food is Milk - Excluding Butter
In Azerbaijan the most eaten food is Wheat and products
In Bahamas the most eaten food is Vegetables, other
In Bahrain the most eaten food is Milk - Excluding Butter
In Bangladesh the most eaten food is Rice and products
In Barbados the most eaten food is Sugar cane
In Belarus the most eaten food is Potatoes and products
In Belgium the most eaten food is Milk - Excluding Butter
In Belize the most eaten food is Sugar cane
In Benin the most eaten food is Yams
In Bhutan the most eaten 

In [ ]:
# Finetuning with our data

# 1. load pretrained Tokenizer, call it with dataset (encode the dataset)
model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

sequence = "This is temporary tokenized dataset"
tokenized_dataset = tokenizer(sequence, return_tensors="pt")
print(tokenized_dataset)

# training_args = TrainingArguments("trainer")

# trainer = Trainer(
#     model,
#     training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["validation"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
# )

# trainer.train()

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[128000,   2028,    374,  13643,   4037,   1534,  10550]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
# Fine-tuned model

classifier = pipeline(
    "text-generation", 
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    max_length=50
)

In [ ]:
# Save model
model.save_pretrained("meta-llama/Llama-3.2-1B")

# Load model
model = AutoModelForSequenceClassification.from_pretrained("meta-llama/Llama-3.2-1B")